In [ ]:
#import packages
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from sklearn.metrics import accuracy_score
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
#mounting google drive on colab notebook
from google.colab import drive
drive.mount('/d')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /d


In [ ]:
#reading data from csv file
data1 = pd.read_csv("/d/My Drive/New_Data_fully_formatted.csv")

In [ ]:
X = data1.loc[:,['GET','POST','POST Body']]
y = data1.loc[:,['Class']]

In [ ]:
#checking the number of classes in the data
class_set = set(y['Class'].tolist())
print(class_set)

{'Attack', 'Valid'}


In [ ]:
label_encoder = LabelEncoder()
y = np.array(label_encoder.fit_transform(y['Class']))

In [ ]:
#fil the nans with 0
X.fillna(0, inplace = True)

#iterate through each row and remove redundant chunks of text from the 3 columns
for i,rows in X.iterrows():
    l = []
    for j in rows:
        #print(j)
        if j!=0:
            if j.startswith('//localhost:8080/tienda1/'):
                j = j[len('//localhost:8080/tienda1/') :]
            if j.endswith(' HTTP/1.1'):
                j = j[:len(j)-len(' HTTP/1.1')]
            l.append(j)
        else:
            l.append(str(j))
   # print(l)
    X.replace(to_replace = rows[0], value = l[0], inplace = True) 
    X.replace(to_replace = rows[1], value = l[1], inplace = True) 
    X.replace(to_replace = rows[2], value = l[2], inplace = True)
    

In [ ]:
X.head(10)

,GET,POST,POST Body
0,0,/rest/user/login,"nanemail:<iframe src=\""javascript:alert(`xss`)..."
1,0,/rest/user/login,nanemail:AND (SELECT * FROM (SELECT(SLEEP(5)))...
2,0,/rest/user/login,"nanemail:UNION ALL SELECT 1,2,3,4,5,6,7,8,9,10..."
3,0,rest/user/login,"nanemail:@@VERSION,SLEEP(5),USER(),BENCHMARK(1..."
4,0,/rest/user/login,"nanemail:UNION SELECT @@VERSION,SLEEP(5),USER(..."
5,0,/rest/user/login,"nanemail:@@VERSION,USER(),SLEEP(5),BENCHMARK(1..."
6,0,/rest/user/login,"nanemail:@@VERSION,USER(),SLEEP(5),BENCHMARK(1..."
7,0,/rest/user/login,"nanemail:UNION ALL SELECT 'INJ'||'ECT'||'XXX',..."
8,0,/rest/user/login,nanemail:VU5JT04gQUxMIFNFTEVDVCAnSU5KJ3x8J0VDV...
9,0,/rest/user/login,nanemail:%55%4e%49%4f%4e%20%41%4c%4c%20%53%45%...


In [ ]:
#the data in both the columns are decoded
from urllib.parse import unquote
for i,rows in X.iterrows():
    if rows[0] != '0' :
        rows[0] = unquote(rows[0])
    if rows[1] != '0':
        rows[1] = unquote(rows[1])
    if rows[2] != '0':
        rows[2] = unquote(rows[2])
  

In [ ]:
vectorizer = CountVectorizer()  
vectorizer1 = CountVectorizer()
vectorizer2 = CountVectorizer()

In [ ]:
vectorizer.fit(X['GET'])
#print(vectorizer.vocabulary_)
vector = vectorizer.transform(X['GET']).toarray()

In [ ]:
vectorizer1.fit(X['POST'])
#print(vectorizer1.vocabulary_)
vector1 = vectorizer1.transform(X['POST']).toarray()

In [ ]:
vectorizer2.fit(X['POST Body'])
#print(vectorizer2.vocabulary_)
vector2 = vectorizer2.transform(X['POST Body']).toarray()

In [ ]:
X_train = np.concatenate((vector,vector1,vector2),axis = 1)
y_train = np.reshape(y,(len(data1,)))


In [ ]:
print(X_train.shape)
print(y_train.shape)


(208, 850)
(208,)


In [ ]:
#splitting data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.20)

In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
model.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)

In [ ]:
print("MLP Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

MLP Classifier :
Accuracy : 0.8571428571428571
Precision Score : 0.8571428571428571
Recall score : 0.8571428571428571
F1 - score : 0.8571428571428571
Confusion Matrix :
 [[23  3]
 [ 3 13]]
Classification Report :
               precision    recall  f1-score   support

      Attack       0.88      0.88      0.88        26
       Valid       0.81      0.81      0.81        16

    accuracy                           0.86        42
   macro avg       0.85      0.85      0.85        42
weighted avg       0.86      0.86      0.86        42



In [ ]:
#Model 1: Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)

In [ ]:
print("Random Forest Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Random Forest Classifier :
Accuracy : 0.9047619047619048
Precision Score : 0.9107142857142857
Recall score : 0.9047619047619048
F1 - score : 0.905658263305322
Confusion Matrix :
 [[23  3]
 [ 1 15]]
Classification Report :
               precision    recall  f1-score   support

      Attack       0.96      0.88      0.92        26
       Valid       0.83      0.94      0.88        16

    accuracy                           0.90        42
   macro avg       0.90      0.91      0.90        42
weighted avg       0.91      0.90      0.91        42



In [ ]:
#model 2 : Logistic Regression
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)
print(accuracy)

0.8809523809523809


In [ ]:
print("Logistic Regression Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Logistic Regression Classifier :
Accuracy : 0.8809523809523809
Precision Score : 0.8832492997198879
Recall score : 0.8809523809523809
F1 - score : 0.8815889992360582
Confusion Matrix :
 [[23  3]
 [ 2 14]]
Classification Report :
               precision    recall  f1-score   support

      Attack       0.92      0.88      0.90        26
       Valid       0.82      0.88      0.85        16

    accuracy                           0.88        42
   macro avg       0.87      0.88      0.88        42
weighted avg       0.88      0.88      0.88        42



In [ ]:
#Model 3: Stochastic Gradient Descent Classifier
from sklearn.linear_model import SGDClassifier
model = SGDClassifier()
model.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
r = model.predict(X_test)
accuracy = accuracy_score(y_true = y_test, y_pred = r)
print(accuracy)

0.9047619047619048


In [ ]:
print("Stochastic Gradient Descent Classifier :")
print("Accuracy :", accuracy)
print("Precision Score :", precision_score(y_test, r, average = 'weighted'))
print("Recall score :", recall_score(y_test, r, average = 'weighted'))
print("F1 - score :", f1_score(y_test, r, average = 'weighted'))
print("Confusion Matrix :\n",confusion_matrix(y_test, r))
print("Classification Report :\n",classification_report(y_test, r, target_names=class_set))

Stochastic Gradient Descent Classifier :
Accuracy : 0.9047619047619048
Precision Score : 0.9047619047619048
Recall score : 0.9047619047619048
F1 - score : 0.9047619047619048
Confusion Matrix :
 [[24  2]
 [ 2 14]]
Classification Report :
               precision    recall  f1-score   support

      Attack       0.92      0.92      0.92        26
       Valid       0.88      0.88      0.88        16

    accuracy                           0.90        42
   macro avg       0.90      0.90      0.90        42
weighted avg       0.90      0.90      0.90        42

